In [1]:
from get_info_followers_and_times import *
from modelo_tweets import *
from classify import *
from get_scores import *
import seaborn as sns

In /u01/user8/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /u01/user8/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /u01/user8/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /u01/user8/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /u01/user8/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-d

In [2]:
class modeloBase:
    
    def train_model(self):
        pass
       
    def predict_model(self):
        pass
    
    def validation_model(self):
        pass
        

In [6]:
class modelPercepcion(modeloBase):
    """
    Clase del modelo de percepcion de seguridad general juntando los procesos de clasificacion de tweets relacionados
    con seguridad, cuantificacion de los tweets clasificados y prediccion de la serie de tiempo en tiempo futuro.
    """    
    def __init__(self,
                 original_data,
                 train_period,
                 val_period,
                 classify_model=predict_from_data,
                 quantify_model=calculate_scores,
                 tweets_model_base=modelTweets,
                 f_covariates=(T_c,restore_date),
                 win_size_for_partition_cov=1,
                 followers_rate=2,
                 win_size_infectious_rate = 3,
                 win_size_train_period = 1,
                 win_size_pred_period = 1,
                 method_pred = 'integral'
                ):
        #original_data
        self.original_data=original_data
        # function to classify tweets related to security
        self.classify_model=classify_model
        if self.classify_model == None:
            pass
            # TODO: funcion para entrenar modelo de clasificacion textos relacionados seguridad
        # function to quantify tweets related to security
        self.quantify_model=quantify_model
        if self.quantify_model == None:
            pass
            # TODO: funcion para entrenar modelo de clasificacion textos relacionados seguridad
            
        # variables proceso de hawkes    
        self.train_period = train_period
        self.validate_period = val_period
        self.tweets_model_base = tweets_model_base
        self.f_covariates = f_covariates
        self.win_size_for_partition_cov=win_size_for_partition_cov
        self.followers_rate = followers_rate
        self.win_size_infectious_rate  = win_size_infectious_rate
        self.win_size_train_period  = win_size_train_period
        self.win_size_pred_period  = win_size_pred_period
        self.method_pred  = method_pred
    
    def prepare_data(self):
        """
        Preprocesamiento de los datos y formateo para que queden listos para la entrada del modelo final
        :return: 
        """
        tweets_predict=self.classify_model(self.original_data)
        sentiment_data=self.quantify_model(pd.read_csv("entradas/ScoreTotalData.csv"))#sentiment_data
        # base de datos usuarios y sus seguidores
        users_data=pd.read_csv("results/users.csv")
        #base de datos reconocimiento de tweets y RT
        rt_data=pd.read_csv("results/tweets_from.csv")
        tweets=build_dict_info_tweets(sentiment_data,users_data,rt_data,tweets_predict)
        
        self.tweets_model = self.tweets_model_base(tweets,
                                                   self.train_period,
                                                   self.validate_period,
                                                   f_covariates=self.f_covariates,
                                                   win_size_for_partition_cov=self.win_size_for_partition_cov,
                                                   followers_rate=self.followers_rate,
                                                   win_size_infectious_rate = self.win_size_infectious_rate,
                                                   win_size_train_period = self.win_size_train_period,
                                                   win_size_pred_period = self.win_size_pred_period,
                                                   method_pred = self.method_pred)
        
    def train_model(self):
        """
        Calcula los parametros ajustados del modelo de series de tiempo
        :return: (Beta,param_infectious_fit)
        """
        self.tweets_model.train()
        return self.tweets_model.Beta, self.tweets_model.param_infectious_fit
        
    def predict_model(self):
        """
        Calcula los errores de prediccion de la serie de tiempo para el tiempo predecido
        :return: (Beta,param_infectious_fit)
        """
        self.tweets_model.compute_lambda_predict()
        return self.tweets_model.Tweets_pred
    
    def validation_model(self):
        """
        Calcula las metricas de error prediccion de serie de tiempo para el tiempo predecido
        :return: diccionario diferentes metricas calculadas
        """
        self.tweets_model.compute_errors()
        return self.tweets_model.errors_predict
        

In [4]:
data=pd.read_csv("entradas/Tweet.csv",delimiter=';')

In [10]:
partidos=pd.read_csv('entradas/Partidos.csv')
manifestaciones=pd.read_csv('entradas/Manifestaciones.csv')
f_especiales=pd.read_csv('entradas/FechasEspeciales.csv')

partidos_M=pd.to_datetime(partidos['Fecha'].dropna())
partidos_S=pd.to_datetime(partidos['Fecha.1'].dropna())
manifestaciones=pd.to_datetime(manifestaciones['Fecha'].dropna())
f_especiales=pd.to_datetime(f_especiales[['2019','2020']].values.flatten())
f_especiales=f_especiales[~f_especiales.isna()]

def TC(t):
    return np.array([
                     t.weekday()/6.0,
                     (t.hour > 12)*1,
                     (pd.Timestamp(t.date()) == partidos_M).sum(),
                     (pd.Timestamp(t.date()) == partidos_S).sum(),
                     (pd.Timestamp(t.date()) == manifestaciones).sum(),
                     (pd.Timestamp(t.date()) == f_especiales).sum(),
                     1
                    ])

In [16]:
M=modelPercepcion(data,
                  ('2019-04-01 00:00', '2019-05-01 00:00'),
                  ('2019-05-01 00:00', '2019-05-10 00:00'),
                  tweets_model_base=modelTweets,
                  f_covariates=(TC,restore_date),
                  win_size_for_partition_cov=1,
                  followers_rate=2,
                  win_size_infectious_rate = 3,
                  win_size_train_period = 1,
                  win_size_pred_period = 1,
                  method_pred = 'integral')

In [19]:
M.prepare_data()
M.tweets_model.f_inicio

Timestamp('2019-03-17 22:46:36')

In [20]:
Beta, param_infectious_fit = M.train_model()
Beta, param_infectious_fit

2.0370537754860713e-09
[28.33333333 30.         18.         16.          0.          6.
 60.        ]
2.0370537754860713e-09


(array([-0.62571055, -0.71687847,  0.21086107, -0.24770673,  0.        ,
        -0.75993047,  2.28682908]),
 array([-2.05564784e-04,  2.68467820e-01,  1.99999873e+01]))

In [22]:
T_pred=M.predict_model()
T_pred

,start,end,Tweets
0,1057.223333,1058.227984,87.595718
1,1058.227984,1059.232636,9.898557
2,1059.232636,1060.237287,7.667656
3,1060.237287,1061.241938,5.505109
4,1061.241938,1062.246589,5.164919
...,...,...,...
210,1268.200078,1269.204729,7.302079
211,1269.204729,1270.209380,7.311694
212,1270.209380,1271.214031,7.232873
213,1271.214031,1272.218682,7.158639


In [23]:
Errors=M.validation_model()
Errors

/u01/user8/Documents/percepcion/limpio/modelo_tweets.py:894: RuntimeWarning: divide by zero encountered in true_divide
  self.errors_predict['APE']=diff/real


{'APE': array([6.96324705e+00, 5.69627971e-01, 3.61028644e-01, 3.11861400e-01,
        2.62154431e-01, 3.58423517e-01, 9.95564453e-02, 5.37731079e-01,
        4.38452080e-01, 7.36227258e-01, 7.72787530e-01, 7.04527664e-01,
        4.85657913e-01, 1.56009517e+00, 2.62969156e-01, 1.45123369e+00,
                   inf,            inf,            inf, 2.10603076e-01,
        6.14181029e-01, 8.19453996e-01, 8.90623864e-01, 8.66144744e-01,
        3.13547094e-01, 7.18868753e-02, 5.02642147e-01, 6.46509715e-01,
        3.46017093e-01, 8.05599067e-02, 1.31339902e-02, 1.23646462e+00,
        4.28684336e-01, 4.95574479e-01, 5.14690644e-01, 2.13268025e-01,
        4.07420770e-01, 2.84904254e-01, 4.40414699e-01, 1.82677877e+00,
        4.74116876e+00,            inf, 4.83091785e+00, 4.69421377e-01,
        5.78686764e-01, 8.03709016e-01, 7.08362258e-01, 7.20318998e-01,
        4.77328482e-01, 3.07473778e-01, 3.49861356e-01, 2.31561469e-01,
        6.86299348e-02, 6.45847679e-02, 5.03467095e-01, 4

In [24]:
Errors["MAE"],Errors["Pearson"]

(8.391366605132118, 0.1583757232202767)

In [ ]:
## Lectura datos tweets 
data=pd.read_csv("entradas/Tweet.csv",delimiter=';')
sentiment_data=pd.read_csv("entradas/ScoreTotalData.csv")
# tweets_predict=pd.read_csv("results/predict_socialmedia.csv")
tweets_predict=predict_from_data(data)
#calculo score de percepcion seguridad
sentiment_data=calculate_scores(sentiment_data)
# sentiment_data=pd.read_csv("results/POS.csv")
# base de datos usuarios y sus seguidores
users_data=pd.read_csv("results/users.csv")
#base de datos reconocimiento de tweets y RT
rt_data=pd.read_csv("results/tweets_from.csv")
tweets=build_dict_info_tweets(sentiment_data,users_data,rt_data,tweets_predict)
# with open("../hawkes/TiDeH-master/data_tweets.pickle", 'rb') as handle:
#     tweets = pickle.load(handle)

In [3]:
print("Datos totales originales:",len(data))
print("Fechas inicial y final:",data.CreatedAt.min(),data.CreatedAt.max())
print("Registros que son RT:",data.IsRetweet.sum())
print("Registos que pasan la clasificación:",len(tweets_predict))
print("Post originales con cuantificación:",len(sentiment_data))
print("Cantidad de usuario en la base de datos:",len(users_data))

Datos totales originales: 1701668
Fechas inicial y final: 2019-03-18 13:17:14.000 2020-04-28 13:06:06.000
Registros que son RT: 1148043
Registos que pasan la clasificación: 106880
Post originales con cuantificación: 26255
Cantidad de usuario en la base de datos: 90780
